# Advanced Lane Finding Project 

__The goals / steps of this project are the following:__ 


1- Compute the camera calibration matrix and distortion coefﬁcients given a set of chessboard images.  
2- Apply a distortion correction to raw images.  
3- Use gradients and color transforms to create a thresholded binary image.  
4- Apply a perspective transform to rectify binary image ("birds-eye view").  
5- Detect lane pixels and ﬁt to ﬁnd the lane boundary.  
6- Determine the curvature of the lane and vehicle position with respect to center.  
7- Warp the detected lane boundaries back onto the original image.  
8- Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.  


In [ ]:
# importing some useful packages
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual

%matplotlib inline

## 1. Calibration of the camera

In [ ]:
def _build_full_paths(dir_root):
    return [dir_root+'/'+fn for fn in os.listdir(dir_root)]

# The process to calibrate the camera is split into 2 phases:
# - first find the corners of several images with different angles of a chessboards,
# - second, calibrate the camera and 
class CameraCalibrator:
    def __init__(self, nx=9, ny=6):
        self.nx = nx
        self.ny = ny
        self.mtx = None
        self.dist = None
        
    def _find_corners(self, filenames, show_images=False):
        objpoints, imgpoints = [], [] # 3d points in real world space, # 2d points in image plane.
    
        # Prepare obj points
        objp = np.zeros((self.nx * self.ny, 3), np.float32)
        objp[:,:2] = np.mgrid[0:self.nx, 0:self.ny].T.reshape(-1,2)
        
        img_size = None
    
        for fn in filenames:
            #reading in an image
            image = mpimg.imread(fn)
            gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

            ret, corners = cv2.findChessboardCorners(gray, (self.nx,self.ny), None)
        
            if img_size is None:
                img_size = (image.shape[1], image.shape[0])
        
            if ret == True:
                objpoints.append(objp)
                imgpoints.append(corners)
                
                if show_images:
                    image = cv2.drawChessboardCorners(image, (self.nx, self.ny), corners, ret)
                    plt.imshow(image)
                    plt.figure()
    
        return objpoints, imgpoints, img_size
    
    def _calibrate_camera(self, objpoints, imgpoints, img_size):
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
        return mtx, dist
    
    def calibrate(self, filenames):
        objpoints, imgpoints, img_size = self._find_corners( filenames, show_images=True )
        self.mtx, self.dist = self._calibrate_camera(objpoints, imgpoints, img_size)
        return self.mtx, self.dist
       
        
ccal1 = CameraCalibrator()
ccal1.calibrate( _build_full_paths('camera_cal/') );


## 2.  Apply a distortion correction to raw images.

### a. Compute undistort image

In [ ]:
def _show_dist_undist_images_by_2(images, title0='Original Image', title1='Undistorted Image'):
    for img, dst in images:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.imshow(img, cmap='gray')
        ax1.set_title(title0, fontsize=30)
        ax2.imshow(dst, cmap='gray')
        ax2.set_title(title1, fontsize=30)

class Undistorter:
    def __init__(self, camera_calibrator):
        self.camera_calibrator = camera_calibrator
        
    def undistort_image(self, img):
        mtx, dist = self.camera_calibrator.mtx, self.camera_calibrator.dist
        return cv2.undistort(img, mtx, dist, None, mtx)
        
    def read_undistort_images(self, filenames):
        undistorted_imgs = []
    
        for fn in filenames:
            img = mpimg.imread(fn)
        
            undst = self.undistort_image(img)
            undistorted_imgs.append( (img, undst, ) )
        
        return undistorted_imgs
        
undistorter = Undistorter(ccal1)


In [ ]:
_show_dist_undist_images_by_2( undistorter.read_undistort_images([ 'camera_cal/calibration1.jpg' ]) )

### b. Compute perspective matrix

In [ ]:
class PerspectiveWarper:
    def __init__(self):
        self.M = None
        self.Minv = None
        
    def _perspective(self, matrix, img):  
        if matrix is None:
            raise ValueError("!!! The matrix M must be computed by method compute_perspective_transform() before calling warp_perspective()")
            
        img_size = (img.shape[1], img.shape[0])
        return cv2.warpPerspective(img, matrix, img_size)
    
    def compute_perspective_transform(self, src, dst):
        self.M = cv2.getPerspectiveTransform(src, dst)
        self.Minv = cv2.getPerspectiveTransform(dst, src)
    
    def warp_perspective(self, img):
        return self._perspective(self.M, img)
    
    def unwarp_perspective(self, img):
        return self._perspective(self.Minv, img)
    


#### Perspective warper applied to chessboards

In [ ]:
chess_persp_warper = PerspectiveWarper()

def chessboard_perspective_warper(perspective_warper, img, offset = 100, nx=9, ny=6, draw_corners=True):
    undist = undistorter.undistort_image(img)
    # Convert undistorted image to grayscale
    gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
    # Search for corners in the grayscaled image
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    
    if ret == True:
        if draw_corners:
            cv2.drawChessboardCorners(undist, (nx, ny), corners, ret)
            
        img_size = (gray.shape[1], gray.shape[0])
        
        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                            [img_size[0]-offset, img_size[1]-offset], [offset, img_size[1]-offset]])
    
        perspective_warper.compute_perspective_transform(src, dst)
        return perspective_warper.warp_perspective(undist)
    
    return None
    
def read_perspective_warper_chess_images(perspective_warper, filenames):
    result = []
    for fn in filenames:
        img = mpimg.imread(fn)
        warped = chessboard_perspective_warper(perspective_warper, img)
        
        if warped is None:
            continue
        result.append( (img, warped,) )
        
    return result

_show_dist_undist_images_by_2( read_perspective_warper_chess_images(chess_persp_warper, _build_full_paths('camera_cal/')), title1='Warped Image' )

#### Perspective warper applied to road images

In [ ]:
persp_warper = PerspectiveWarper()

# Compute the perspective matrix for the road:
def compute_road_perspective_matrix(perspective_warper):
    # src = np.float32([[535, 460], [750, 460], [1480, 720], [-180, 720]])
    src = np.float32([[535, 460], [750, 460], [1480, 720], [-180, 720]])
    dst = np.float32([[0, 0], [1280, 0], [1280, 720], [0, 720]])
    
    perspective_warper.compute_perspective_transform(src, dst)

compute_road_perspective_matrix(persp_warper)
    
def read_perspective_warper_road_images(warp_perspective_fct, filenames):
    result = []
    for fn in filenames:
        img = mpimg.imread(fn)
        undist = undistorter.undistort_image(img)
        warped = warp_perspective_fct(undist)
        
        if warped is None:
            continue
        
        result.append( (img, warped,) )
        
    return result

_show_dist_undist_images_by_2( read_perspective_warper_road_images(persp_warper.warp_perspective, _build_full_paths('test_images/')), title1='Warped Image' )

In [ ]:
img = mpimg.imread('test_images/straight_lines1.jpg')
undist = undistorter.undistort_image(img)

def find_perspective_matrix(xt1,xt2,yt,xl1,xl2,yl):
    # src = np.float32([[535, 460], [750, 460], [1480, 720], [-180, 720]])
    src = np.float32([[xt1, yt], [xt2, yt], [xl2, yl], [xl1, yl]])
    dst = np.float32([[0, 0], [1280, 0], [1280, 720], [0, 720]])
    persp_warper.compute_perspective_transform(src, dst)
    
    warped = persp_warper.warp_perspective(undist)
    
    _show_dist_undist_images_by_2([(img, warped)])
    plt.show()
    
interact(find_perspective_matrix, 
            xt1=widgets.IntSlider(min=0,max=650,step=1,value=535),
            xt2=widgets.IntSlider(min=630,max=1280,step=1,value=750),
            yt=widgets.IntSlider(min=360,max=720,step=1,value=460),
            xl1=widgets.IntSlider(min=-300,max=650,step=1,value=-180),
            xl2=widgets.IntSlider(min=600,max=1550,step=1,value=1480),
            yl=widgets.IntSlider(min=360,max=800,step=1,value=720),
        )

## 3. Use gradients and color transforms to create a thresholded binary image.

In [ ]:
class ImageSobelThresholder:
    def __init__(self, sobel_kernel=9, abs_thresh=(20,100), mag_thresh=(30, 100), dir_thresh=(0.7, 1.3)):
        self.sobel_kernel = sobel_kernel
        
        # Absolute sobelthreshold
        self.abs_thresh = abs_thresh
        self.mag_thresh = mag_thresh
        self.dir_thresh = dir_thresh
        
    def abs_sobel_threshold(self, gray, orient='x'):
        # Take the derivative in x or y given orient = 'x' or 'y'
        if orient == 'x':
            sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=self.sobel_kernel)
        elif orient == 'y':
            sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=self.sobel_kernel)
    
        # Take the absolute value of the derivative or gradient
        abs_sobel = np.absolute(sobel)
    
        # Scale to 8-bit (0 - 255) then convert to type = np.uint8
        scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
        # Create a mask of 1's where the scaled gradient magnitude is > thresh_min and < thresh_max
        binary_output = np.zeros_like(scaled_sobel)
        binary_output[(scaled_sobel >= self.abs_thresh[0]) & (scaled_sobel <= self.abs_thresh[1])] = 1

        # Return this mask as your binary_output image
        return binary_output
    
    
    def mag_sobel_threshold(self, gray):
        # Take both Sobel x and y gradients
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=self.sobel_kernel)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=self.sobel_kernel)
        # Calculate the gradient magnitude
        gradmag = np.sqrt(sobelx**2 + sobely**2)
        # Rescale to 8 bit
        scale_factor = np.max(gradmag)/255. 
        gradmag = (gradmag/scale_factor).astype(np.uint8) 
        # Create a binary image of ones where threshold is met, zeros otherwise
        binary_output = np.zeros_like(gradmag)
        binary_output[(gradmag >= self. mag_thresh[0]) & (gradmag <= self.mag_thresh[1])] = 1

        # Return the binary image
        return binary_output
    
    
    def dir_sobel_threshold(self, gray):
        # Calculate the x and y gradients
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=self.sobel_kernel)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=self.sobel_kernel)
        # Take the absolute value of the gradient direction, 
        # apply a threshold, and create a binary image result
        absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
        binary_output =  np.zeros_like(absgraddir)
        binary_output[(absgraddir >= self.dir_thresh[0]) & (absgraddir <= self.dir_thresh[1])] = 1

        # Return the binary image
        return binary_output
    
    
    def combined_sobel_thresholds(self, gray):
        # Apply each of the thresholding functions
        gradx = self.abs_sobel_threshold(gray, orient='x')
        grady = self.abs_sobel_threshold(gray, orient='y')
        mag_binary = self.mag_sobel_threshold(gray)
        dir_binary = self.dir_sobel_threshold(gray)
    
        combined = np.zeros_like(dir_binary)
        combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
        return combined
    
    def gray_combined_sobel_thresholds(self, undist):
        gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
        return self.combined_sobel_thresholds(gray)

In [ ]:
def apply_thresholds(undist, threshold_fct, cvt_2_gray=True):
    if cvt_2_gray:
        gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
        return threshold_fct(gray)
    else:
        return threshold_fct(undist)

def show_thresholded_binary_images(undistorter, threshold_fct, cvt_2_gray=True):
    result = []
    for fn in _build_full_paths("test_images/"):
        img = mpimg.imread(fn)
        undist = undistorter.undistort_image(img)
        
        # undist = persp_warper.warp_perspective(undist)
        
        thres_img = apply_thresholds(undist, threshold_fct, cvt_2_gray)
        result.append( (img, thres_img,) )
        
    return result

sobel_thresholder = ImageSobelThresholder()

### a. Apply Gradients

#### Sobel

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, sobel_thresholder.abs_sobel_threshold), title1='Sobel Binary Image' )

#### Gradient Magnitude

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, sobel_thresholder.mag_sobel_threshold), title1='Gradient Magnitude Image' )


#### Gradient direction

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, sobel_thresholder.dir_sobel_threshold), title1='Gradient Direction Binary Image' )

#### Combined gradient thresholds

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, sobel_thresholder.combined_sobel_thresholds), title1='Combined Thresholded Binary Image' )


In [ ]:
img = mpimg.imread('test_images/test1.jpg')
undist = undistorter.undistort_image(img)
undist = persp_warper.warp_perspective(undist)
gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)

def find_gradient_thresholds(sobel_kernel, abs_thresh_low, abs_thresh_high, mag_thresh_low, mag_thresh_high, dir_thresh_low, dir_thresh_high):
    sobel_kernel = int(sobel_kernel)
    sobel_thresholder = ImageSobelThresholder(sobel_kernel=sobel_kernel, abs_thresh=(abs_thresh_low,abs_thresh_high), mag_thresh=(mag_thresh_low, mag_thresh_high), dir_thresh=(dir_thresh_low, dir_thresh_high))
    binary = sobel_thresholder.combined_sobel_thresholds(gray)
    
    _show_dist_undist_images_by_2([(img, binary)])
    plt.show()
    
interact(find_gradient_thresholds, 
            sobel_kernel=widgets.Dropdown(options=['3', '5', '7', '9', '13', '15', '17', '21', '23', '25', '27'], value='9'),
            abs_thresh_low=widgets.IntSlider(min=0,max=255,step=1,value=25),
            abs_thresh_high=widgets.IntSlider(min=0,max=255,step=1,value=100),
            mag_thresh_low=widgets.IntSlider(min=0,max=255,step=1,value=30),
            mag_thresh_high=widgets.IntSlider(min=0,max=255,step=1,value=100),
            dir_thresh_low=widgets.FloatSlider(min=0,max=3.2,step=0.05,value=1.6),
            dir_thresh_high=widgets.FloatSlider(min=0,max=3.2,step=0.05,value=3.2),
        )

### b. Apply Color Transforms

#### Color Threshold

In [ ]:
class ImageColorThresholder:
    def __init__(self, ch_thresh=( ('H', (15, 100)), ('S', (90, 255)) ) ):
        self.ch_thresh = ch_thresh
        
    @staticmethod
    def color_threshold(img, channel='S', thresh=(90, 255)):
        ch_map = {
            'H': 0,
            'L': 1,
            'S': 2
        }
        hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        channel = hls[:,:,ch_map[channel]]
        
        binary = np.zeros_like(channel)
        binary[(channel > thresh[0]) & (channel <= thresh[1])] = 1
        return binary
    
    def combined_color_thresholds(self, img):
        prev_binary = None
        for ch, thresh in self.ch_thresh:
            binary = self.color_threshold(img, ch, thresh)
            if prev_binary is not None:
                binary = binary | prev_binary
            
            prev_binary = binary
    
        img_size = (img.shape[1], img.shape[0])
        result = np.zeros_like(img_size)
        result[binary] = 1
        return binary
    
color_thresholder = ImageColorThresholder()

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, ImageColorThresholder.color_threshold, cvt_2_gray=False), title1='Color Transform' )


In [ ]:
def test_L_color_thresholding(img):
    return ImageColorThresholder.color_threshold(img, channel='S', thresh=(90, 255))

_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, test_L_color_thresholding, cvt_2_gray=False), title1='Color Transform' )


#### Combined Color Threshold

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, color_thresholder.combined_color_thresholds, cvt_2_gray=False), title1='Combined Color Transform' )


In [ ]:
img = mpimg.imread('test_images/test5.jpg')
undist = undistorter.undistort_image(img)

def find_color_thresholds(H_thresh, S_thresh):
    color_thresholder = ImageColorThresholder(ch_thresh = (('H', H_thresh,), ('S', S_thresh,),) )
    binary = color_thresholder.combined_color_thresholds(undist)
    
    _show_dist_undist_images_by_2([(img, binary)])
    plt.show()
    
interact(find_color_thresholds,
            H_thresh=widgets.FloatRangeSlider(value=[15, 100], min=0, max=180, step=1,),
            # L_thresh=widgets.FloatRangeSlider(value=[2, 20], min=0, max=255, step=1,),
            S_thresh=widgets.FloatRangeSlider(value=[90, 255], min=0, max=255, step=1,),
        )

## 4. 

In [ ]:
def image_processing_pipeline(undistorter, perspective_warper, sobel_thresholder, color_thresholder, img):
    undist = undistorter.undistort_image(img)
    warped = perspective_warper.warp_perspective(undist)
    
    color_binary = color_thresholder.color_threshold(warped)
    sobel_binary = sobel_thresholder.combined_sobel_thresholds(color_binary)
    
    binary = np.zeros_like(color_binary)
    binary[(sobel_binary == 1) | (color_binary == 1)] = 1
    
    return undist, warped, binary

In [ ]:
class LanesFinder:
    def __init__(self, nb_window=15, margin=100, minpix=20):
        self.nb_window = nb_window
        self.margin = margin
        self.minpix = minpix
    
    def find_lanes(self, binary_warped, draw_window=False):
        def measure_curvature(ploty, left_fitx, right_fitx):
            y_eval = np.max(ploty)
            # Define conversions in x and y from pixels space to meters
            ym_per_pix = 30/720 # meters per pixel in y dimension
            xm_per_pix = 3.7/700 # meters per pixel in x dimension

            # Fit new polynomials to x,y in world space
            left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
            right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
            # Calculate the new radii of curvature
            left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
            right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
            # Now our radius of curvature is in meters
            return left_curverad, right_curverad
        
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        left_lane_inds, right_lane_inds = [], []
        
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/self.nb_window)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        leftx_current, rightx_current = None, None
        
        for idx, window in enumerate(range(self.nb_window)):
            win_from = np.int((idx+1) * window_height)
            histogram = np.sum(binary_warped[win_from:,:], axis=0)
            
            midpoint = np.int(histogram.shape[0]/2)
            if leftx_current is None:
                leftx_current = np.argmax(histogram[:midpoint])
                
            if rightx_current is None:
                rightx_current = np.argmax(histogram[midpoint:]) + midpoint
            
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - self.margin
            win_xleft_high = leftx_current + self.margin
            win_xright_low = rightx_current - self.margin
            win_xright_high = rightx_current + self.margin
            
            # Draw the windows on the visualization image
            if draw_window:
                cv2.rectangle(out_img,(win_xleft_low, win_y_low),(win_xleft_high, win_y_high),(0,255,0), 2) 
                cv2.rectangle(out_img,(win_xright_low, win_y_low),(win_xright_high, win_y_high),(0,255,0), 2) 
            
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > self.minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > self.minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        
        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        if lefty.size == 0 or righty.size == 0:
            return None, None, None, None, None
    
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
    
        # Visualize the draw
        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        left_curverad, right_curverad = measure_curvature(ploty, left_fitx, right_fitx)
        
        return ploty, left_fitx, right_fitx, left_curverad, right_curverad

In [ ]:
lanes_finder = LanesFinder()

def process_image(img):
    def putText(img, title, location):
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, title, location, font, 1,(255,255,255), 3,cv2.LINE_AA)
    
    undist, warped, binary = image_processing_pipeline(undistorter, persp_warper, sobel_thresholder, color_thresholder, img)
    
    if binary is None:
        return img
    
    ploty, left_fitx, right_fitx, left_curverad, right_curverad = lanes_finder.find_lanes(binary)
    
    if ploty is None:
        return img
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
 
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp =  persp_warper.unwarp_perspective(color_warp)
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    putText(result, 'Left  curve: %.2f m'%(left_curverad), (10,60))
    putText(result, 'Right curve: %.2f m'%(right_curverad), (10,110))
    
    return result


for fn in _build_full_paths('test_images/'):
    img = mpimg.imread(fn)
    result = process_image(img)
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.imshow(result)
    

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

project_output = 'project.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) 

%time white_clip.write_videofile(project_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))

In [ ]:
challenge_output = 'challenge.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(process_image) 

%time white_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))